# 05 오차역전파법

# 5.1 계산 그래프

### 5.1.1 계산 그래프로 풀다

* 계산 그래프를 이용한 문제풀이의 흐름
* 1. 계산 그래프를 구성한다. 
* 2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다. 

* 순전파: 계산을 왼쪽에서 오른쪽으로 진행
* 역전파: 게산을 오른쪽에서 왼쪽으로 진행

### 5.1.2 국소적 계산
* 국소적 계산: 전체에서 어떤 일이 벌어지든 상관없이 자신과 관계된 정보만으로 결과를 출력할 수 있다. 
* 국소적인 계산은 단순하지만, 그 결과를 전달함으로써 전체를 구성하는 복잡한 계산을 해낼 수 있습니다. 

### 5.1.3 왜 계산 그래프로 푸는가
* 계산 그래프의 이점:
* 1. 국소적 계산 - 아무리 복잡해도 각 노드에서는 단순한 계산에 집중하여 문제를 단순화할 수 있다. 
* 2. 계산 그래프는 중간 계산 결과를 모두 보관할 수 있다. 
* 3. 역전파를 통해 미분을 효율적으로 계산할 수 있다. 

# 5.2 연쇄법칙
* 연쇄법칙: 국소적 미분을 전달하는 원리는 연쇄법칙에 따른 것이다. 

### 5.2.1 계산 그래프의 역전파
* 순방향과는 반대 방향으로 국소적 미분을 곱한다. 
* 역전파의 계산 순서로 목표로 하는 미분 값을 효율적으로 구할 수 있다. 

### 5.2.2 연쇄법칙이란?
* 합성 함수: 여러 함수로 구성된 함수
* 연쇄법칙의 원리: "합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다."

### 5.2.3 연쇄법칙과 계산 그래프

# 5.3 역전파
### 5.3.1 덧셈 노드의 역전파
* 덧셈 노드의 역전파는 입력 값을 그대로 흘러보낸다. 
* 덧셈 노드의 역전파는 1을 곱하기만 할 뿐, 입력된 값을 그대로 다음 노드로 보내게 된다. 

### 5.3.2 곱셉 노드의 역전파
* 곱셉 노드 역전파는 상류의 값에 순전파 때의 입력 신호들을 서로 바꾼 값을 곱해서 하류로 보낸다. 

# 5.4 단순한 계층 구현하기
### 5.4.1 곱셉 계층
* forward()는 순전파
* backward()는 역전파

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다. 
        dy = dout * self.x
        
        return dx, dy

* __init__() -> 인스턴스 변수인 x와 y를 초기화한다. 
* forward()에서 x와 y를 인수로 받고 두 값을 곱해서 반환한다. 
* backward()에서 상류에서 넘어온 미분을 순전파 때의 값을 서로 바꿔 곱한 후 하류로 흘린다. 

In [2]:
    apple = 100
    apple_num = 2
    tax = 1.1

    # 계층들
    mul_apple_layer = MulLayer()
    mul_tax_layer = MulLayer()

    # 순전파
    apple_price = mul_apple_layer.forward(apple, apple_num)
    price = mul_tax_layer.forward(apple_price, tax)

    print(price)  # 220.0

220.00000000000003


In [3]:
    # 역전파
    dprice = 1
    dapple_price, dtax = mul_tax_layer.backward(dprice)
    dapple, dapple_num = mul_apple_layer.backward(dapple_price)

    print(dapple, dapple_num, dtax)  # 2.2 110.0 200

2.2 110.00000000000001 200


### 5.4.2 덧셈 계층

In [4]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

* __init__() -> 인스턴스 변수인 x와 y를 초기화할 필요가 없으니 아무 일도 핮 않는다. 
* forward()에서 x와 y를 인수로 받고 두 값을 더해서 반환한다. 
* backward()에서 상류에서 넘어온 미분을 그대로 하류로 흘린다. 

In [5]:
    orange = 150
    orange_num = 3

    # 계층들
    mul_apple_layer = MulLayer()
    mul_orange_layer = MulLayer()
    add_apple_orange_layer = AddLayer()
    mul_tax_layer = MulLayer()

    # 순전파
    apple_price = mul_apple_layer.forward(apple, apple_num)
    orange_price = mul_orange_layer.forward(orange, orange_num)
    all_price = add_apple_orange_layer.forward(apple_price, orange_price)
    price = mul_tax_layer.forward(all_price, tax)

    print(price)  # 715.0

    # 역전파
    dprice = 1
    dall_price, dtax = mul_tax_layer.backward(dprice)
    dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
    dornage, dorange_num = mul_orange_layer.backward(dorange_price)
    dapple, dapple_num = mul_apple_layer.backward(dapple_price)

    print(dapple_num, dapple, dornage, dorange_num, dtax)
    # 110.0 2.2 3.3 165.0 650

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


# 5.5 활성화 함수 계층 구현하기

### 5.5.1 ReLU 계층

In [6]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

* Relu 클래스는 mask라는 인스턴스 변수를 가진다.
* mask는 True/False로 구성된 넘파이 배열이다. 
* 원소 값이 0 이하인 인덱스는 True, 그 외는 False로 유지한다. 

In [7]:
    import numpy as np
    x = np.array([[1.0, -0.5], [-2.0, 3.0]])
    print(x)
    """
    [[ 1.  -0.5]
     [-2.   3. ]]
    """

    mask = (x <= 0)
    print(mask)
    """
    [[False  True]
     [ True False]]
    """

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


'\n[[False  True]\n [ True False]]\n'

### 5.5.2 Sigmoid 계층

* 1단계; 역전파 때는 상류에서 흘러온 값에 순전파의 출력을 제곱한 후 마이너스를 붙이고 곱해서 하류로 전달한다. 
* 2단계: + 노드는 상류의 값을 여과 없이 하류로 내보낸다. 
* 3단계; exp 노드는 y = exp(x) 연산을 수행한다. 
* 4단계: 'x'노드는 순전파 때의 값을 서로 바꿔 곱한다. 

In [8]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

* 순전파의 출력을 인스턴스 변수 out에 보관했다가, 역전파 계산 떄 사용한다. 

# 5.6 Affine/Softmax 계층 구현하기
### 5.6.1 Affine 계층

In [9]:
    X = np.random.rand(2)     # 입력
    W = np.random.rand(2, 3)  # 가중치
    B = np.random.rand(3)     # 편향

    print(X.shape)  # (2,)
    print(W.shape)  # (2, 3)
    print(B.shape)  # (3,)

    Y = np.dot(X, W) + B

(2,)
(2, 3)
(3,)


* 어파인 변환: 신경망의 순전파 때 수행하는 행렬의 곱

### 5.6.2 배치용 Affine 계층

In [10]:
    X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
    B = np.array([1, 2, 3])
    print(X_dot_W)
    """
    [[ 0  0  0]
     [10 10 10]]
    """
    print(X_dot_W + B)
    """
    [[ 1  2  3]
     [11 12 13]]
    """

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


'\n[[ 1  2  3]\n [11 12 13]]\n'

In [11]:
    dY = np.array([[1, 2, 3], [4, 5, 6]])
    print(dY)
    """
    [[1 2 3]
     [4 5 6]]
    """
    dB = np.sum(dY, axis=0)
    print(dB)  # [5 7 9]

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [12]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

### 5.6.3 Softmax-with-Loss 계층
* 소프트맥스 함수는 입력 값을 정규화하여 출력한다.
* 신경망에서 수행하는 작업은 학습과 추론 2 가지입니다. 
* 추론: 일반적으로 Softmax 계층을 사용하지 않는다. 

In [13]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)  # 오버플로 대책
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y


def cross_entropy_error(y, t):
    delta = 1e-7  # 0일때 -무한대가 되지 않기 위해 작은 값을 더함
    return -np.sum(t * np.log(y + delta))


class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None  # 손실
        self.y = None     # softmax의 출력
        self.t = None     # 정답 레이블(원-핫 벡터)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)  # 3.5.2, 4.2.2에서 구현
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = self.y - self.t / batch_size

        return dx


# 5.7 오차역전파법 구현하기

### 5.7.1 신경망 학습의 전체 그림
* 전제: 신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 저정하는 과정을 '학습'이라 합니다. 신경망 학습은 다음과 같이 4단계로 수행합니다. 
* 1단계 - 미니배치: 훈련 데이터 중 일부를 무작위로 가져옵니다. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실 함수 값을 줄이는 것이 목표입니다. 
* 2단계 - 기울기 산출: 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구합니다. 기울기는 손실 함수이ㅡ 값을 가장 작게 하는 방향을 제시합니다. 
* 3단계- 매개변수 갱신: 가중치 매개변수를 기울기 방향으로 아주 조금 갱신합니다. 
* 4단계 - 반복: 1~3단계를 반복합니다. 

### 5.7.2 오차역전파법을 적용한 신경망 구현하기
* TwoLayerNet 클래스의 인스턴스 변수
* params: 딕셔너리 변수로, 신경망의 매겨변수를 보관
* layers: 순서가 있는 딕셔너리 변수로, 신경망의 계층을 보관
* lastLayer: 신경망의 마지막 계층

* TwoLayerNet 클래스의 메서드
* __init__(self, input_size, hidden_size, output_size, weight_init_std): 초기화를 수행한다. 
* predict(self,x): 예측(추론)을 수행한다. 
* loss(self,x,t): 손실 함수의 값을 구한다. 
* accuracy(self,x,t): 정확도를 구한다. 
* numerical_gradient(self,x,t): 가중치 매개변수의 기울기를 수치 미분 방식으로 구한다. 
* gradient(self,x,t): 가중치 매개변수의 기울기를 오차역전파법으로 구한다. 

In [14]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size,
        weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = \
            Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = \
            Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads

### 5.7.3 오차역전파법으로 구한 기울기 검증하기
* 수치 미분의 이점은 구현하기 쉽다. 
* 수치 미분의 구현에는 버그가 숨어 있기 어려운 반면, 오차 역전 파법은 구현하기 복잡해서 종종 실수를 하곤 합니다. 
* 수치 미분의 결과와 오차역전파법의 결과를 비교하여 오차역전파법을 제대로 구현했는지 검증하고 한다. 
* 기울기 확인: 두 방식으로 구한 기울기가 일치함을 확인하는 작업

In [ ]:
(x_train, t_train), (x_test, t_test) = \
        load_mnist(normalize=True, one_hot_label=True)

    network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

    x_batch = x_train[:3]
    t_batch = t_train[:3]

    grad_numerical = network.numerical_gradient(x_batch, t_batch)
    grad_backprop = network.gradient(x_batch, t_batch)
    
    # 각 가중치의 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 낸다.
    for key in grad_numerical.keys():
        diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
        print(key + ":" + str(diff))

### 5.7.4 오차역전파법을 사용한 학습 구현하기

In [ ]:
import sys
import os
import numpy as np
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼 파라메터
iters_num = 10000  # 반복횟수
train_size = x_train.shape[0]
batch_size = 100  # 미니배치 크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    # print(i)
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 오차역전파법으로 기울기 계산
    grad = network.gradient(x_batch, t_batch)

    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    # 1에폭 당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

# 5.8 정리
* 계산 그래프를 이용하면 계산 과정을 시각적으로 파악할 수 있다. 
* 계산 그래프의 노드는 국소적 계산으로 구성된다. 국소적 계산을 조합해 전체 계산을 구성한다. 
* 계싼 그래프의 순전파는 통상의 계산을 수행한다. 한편, 계산 그래프의 역전파로는 각 노드의 미분을 구할 수 있다. 
* 신경망의 구성 요소를 계층으로 구현하여 기울기를 효율적으로 계산할 수 있다(오차역전파법). 
* 수치 미분과 오차역전파법의 결과를 비교하면 오차역전파법의 구현에 잘못이 없는지 확인할 수 있다(기울기 확인).